In [ ]:
# importamos las cosas que vamos a usar de aima
from search import *

class Ascensores(Problem):
    
    def __init__(self,initial, goal):
        
        #Problem.__init__(self, initial, goal)        
        self.initial = initial
        self.goal = goal
        

    def actions(self,estado):
        personas = estado[0]
        ascensores = estado[1]
        acciones = list() 

        for i in range (len(ascensores)):
            ascensor = ascensores[i]
            personaEnAscensor1 = ascensor[2]
            personaEnAscensor2 = ascensor[3]
            
            bloque = i
            if i == 2:
                bloque -= 1
                
            aumentoDeBloque = (ascensor[0] == 4 and bloque == 0) or (ascensor[0] == 8 and bloque == 1) # bloque +1
            descensoBloque = (ascensor[0] == 4 and bloque == 1) or (ascensor[0] == 8 and bloque == 2)#bloqe -1 

            if ascensor[0] in(0, 4, 8): # si el ascensor sube no permite que suba mas y le hace bajar una planta
                if (ascensor[0] == 4 and bloque == 0) or (ascensor[0] == 8 and bloque == 1):
                    acciones.append(("Bajar",personaEnAscensor1 ,personaEnAscensor2,i, bloque))
                if (ascensor[0] == 0 and bloque == 0) or (ascensor[0] == 4 and bloque == 1) or (ascensor[0] == 8 and bloque == 2):
                    acciones.append(("Subir",personaEnAscensor1 , personaEnAscensor2,i, bloque)) 
            else:
                acciones.append(("Subir",personaEnAscensor1 , personaEnAscensor2,i, bloque))
                acciones.append(("Bajar",personaEnAscensor1 , personaEnAscensor2,i, bloque)) 

            for j in range (len(personas)):
                persona = personas[j]
                mismoBloque = (persona[0] == ascensor[0]) and (persona[1] == bloque)#comprueba que estan en la misma planta y bloque
                personaEnAscenor = (j == personaEnAscensor1 or j == personaEnAscensor2) #Esta en el ascensor subido 
                haySitio = (personaEnAscensor1 == -1 or personaEnAscensor2 == -1)#Comprueba que hay plazas libres en el ascensor 

                if mismoBloque == True: # ascensor y persona en la misma planta
                    if (personaEnAscenor == False) and (haySitio == True):# hay sitio y la persona no esta subida
                        acciones.append(("Montar",j , -1,i, bloque)) #sube a la persona i al ascensor j
                    if personaEnAscenor == True:
                        if persona[0] in (4,8):
                            if aumentoDeBloque == True: 
                                bloque += 1
                            if descensoBloque == True: 
                                bloque -= 1
                        acciones.append(("Salir",j ,-1,i, bloque)) #sale la persona j del ascensor i
        return acciones
        
    #accion = (plantaPersona,PlantaAscensor, accion en string)
    def result(self,estado, accion):

        l = list(estado)
        listaPersonas = list(estado[0]) #la primera(0)posicion de la tupla de estado la convertimos a una lista
        listAscensores = list(estado[1])# Idema para los ascensores con la segunda posicion (1)
        perAcc1 = accion[1]
        perAcc2 = accion[2]
        ascensorAcc = accion[3]
        bloqueAcc = accion[4]
        ascensor = list(listAscensores[ascensorAcc])
        planta = ascensor[0]
        

        if accion[0] == "Subir":
            if perAcc1 != -1 :
                persona1 = list(listaPersonas[perAcc1])
                persona1[0] += 1
                listaPersonas[perAcc1] = tuple(persona1)
            if perAcc2 != -1:
                persona2 = list(listaPersonas[perAcc2])
                persona2[0] += 1
                listaPersonas[perAcc2] = tuple(persona2)
            ascensor[0] += 1    
        if accion[0] == "Bajar":
            if perAcc1 != -1:
                persona1 = list(listaPersonas[perAcc1])
                persona1[0] -= 1
                listaPersonas[perAcc1] = tuple(persona1)
            if perAcc2 != -1:
                persona2 = list(listaPersonas[perAcc2])
                persona2[0] -= 1
                listaPersonas[perAcc2] = tuple(persona2)
            ascensor[0] -= 1
        if accion[0] == "Salir":
            if perAcc1 != -1:
                persona1 = list(listaPersonas[perAcc1])
                persona1[2] = -1 # sacamos a la persona del ascensor
                persona1[1] = bloqueAcc
                
                if perAcc1 == ascensor[2]: 
                    ascensor[2] = - 1 # la persona 1 que esta en la posicion 2 de la tupla ascensor la modificamos a -1 asi decimos que no esta montada 
                elif ascensor[3] == perAcc1:
                    ascensor[3] == - 1
                if perAcc1 != -1:
                    listaPersonas[perAcc1] = tuple(persona1)
        if accion[0] == "Montar":
            if ascensor[2] == -1: 
                ascensor[2] = perAcc1 # la persona sube dentro del ascensor
                persona1 = list(listaPersonas[perAcc1])
                persona1[2] = ascensorAcc # le asignameos el ascensor a la persona 
                listaPersonas[perAcc1] = tuple(persona1)
            elif ascensor[3] == -1: 
                ascensor[3] = perAcc1 # la persona sube dentro del ascensor
                persona1 = list(listaPersonas[perAcc1])
                persona1[2] = ascensorAcc # le asignameos el ascensor a la persona 
                listaPersonas[perAcc1] = tuple(persona1)  
        
        listAscensores[ascensorAcc] = tuple(ascensor)
        return (tuple(listaPersonas), tuple(listAscensores))
    
    def goal_test(self, estado): # Devuelve true si se ha alcanzado el estado objetivo
        personas = estado[0]
        objetivo = True

        for i in range (len(personas)): 
            persona = personas[i]
            personaGoal = self.goal[i]

            sonIfuales = (persona[0] == personaGoal[0]) and (persona[1] == personaGoal[1])

            objetivo = objetivo and sonIfuales

        return objetivo

In [ ]:

state_ini = (
   ((2,0,-1), (4,1,-1),(1,0,-1),(8,1,-1),(1,0,-1)), 
    ((1,0,-1,-1), (4,1,-1,-1), (4,1,-1,-1), (8,2,-1,-1))
)

state_fin = ((3, 0, -1), (11, 2, -1), (12, 2, -1), (1, 0, -1), (9, 2, -1))

In [202]:
i = (
   ((1,0,-1), (4,1,-1)), 
    ((1,0,-1,-1), (4,1,-1,-1))
)

fi = ( (3, 0, -1), (7, 1, -1))

In [282]:
i2 = (
   ((1,0,-1), (4,1,-1), (1,0,-1)), 
    ((1,0,-1,-1), (4,1,-1,-1))
)

fi2 = ((12, 2, -1), (11, 2, -1), (9, 2, -1))

In [283]:
i3 = (
   ((1,0,-1), (4,1,-1)), 
    ((1,0,-1,-1), (4,1,-1,-1))
)

fi3 = ((12, 2, -1), (7, 2, -1))

In [284]:
import time
i4 = (
   ((1,0,-1), (4,1,-1)), 
    ((1,0,-1,-1), (4,1,-1,-1))
)

fi4 = ( (5, 1, -1), (8, 2, -1))

In [285]:
p = Ascensores(i2, fi2)
p.initial

(((1, 0, -1), (4, 1, -1), (1, 0, -1)), ((1, 0, -1, -1), (4, 1, -1, -1)))

In [286]:
p.actions(p.initial)

[('Subir', -1, -1, 0, 0),
 ('Bajar', -1, -1, 0, 0),
 ('Montar', 0, -1, 0, 0),
 ('Montar', 2, -1, 0, 0),
 ('Subir', -1, -1, 1, 1),
 ('Montar', 1, -1, 1, 1)]

In [287]:
#p.result(p.initial, ("Subir", 1, -1, 1, 1))

In [288]:
p.goal

((12, 2, -1), (11, 2, -1), (9, 2, -1))

In [289]:
# Importamos los modulos pertenecinetes a los algoritmos de busqueda 
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [292]:
def distanciaObjetivo(node):
    result = 0
    goal = fin
    personas = node.state[0]
    
    for i in range (len(personas)):
        per = personas[i]
        perObj = goal[i]

        result += abs(per[0] - perObj[0])
        result += abs(per[1] - perObj[1])
        
        
    return result

In [293]:
start_time = time.time() #Compute time of execution
best_first_graph_search(p, distanciaObjetivo).solution()
print("--- %s seconds ---" % (time.time() - start_time))

KeyboardInterrupt: 

In [281]:
start_time = time.time() #Compute time of execution
astar_search(p, distanciaObjetivo).solution()
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.2759220600128174 seconds ---


In [131]:
def distanciaPersonaAscensorLibre(node): # busca el ascensor mas cercano y calcula la distancia 
    result = 0 
    
    personas = node.state[0]
    ascensores = node.state[1]
    
    for i in range (len(personas)):
        persona = personas[i]
   
        for j in range (len(ascensores)):
            a = ascensores[j]
            
            estaVacio = a[1][0] == -1 and a[1][1] == -1
            
            if persona[2] == -1: 
            
                if j == 1 or j == 2:
                    if persona[1] == 1: 
                        if estaVacio == True:
                            result += abs(persona[0] - a[0])

                    elif persona[1] == 2:
                        if estaVacio == True:
                            result += abs(persona[0] - a[0])

                if j == persona[1]:
                    if estaVacio == True:
                            result += abs(persona[0] - a[0])
    
    return result
#TODO ver distancia al ascensor so la persona no ha llehado a su objetivo 


In [194]:
#Definimos algunas Hehuristicas 

def linear(node): # dado un estado nos devuelve las personas que todavia no estan en su planta objetivo
# es decir, que estan mal colocadas (como en el puzzle de 8)
    result = 0
    goal = fin
    estado = node.state[0]
    
    for i in range (len(estado)):
        if goal[i] != estado[i]:
            result += 1
    return result

# Sería como el de distancia hasta su objetivo quitando los if que comprueban las paradas para cambio de ascensor
def paradasTotales(node):
    result = 0
    bloque1 = (0,1,2,3,4)
    bloque2 = (4,5,6,7,8)
    bloque3 = (8,9,10,11,12)
    
    goal = fin
    estado = node.state[0]
    
    for i in range (len(estado)):
        plantaIni = estado[i]
        plantaFin = goal[i]
        #podemos ignorar la comporacion de goal[i] == estado[i] ya que el resultado es 0
        result += abs(goal[i] - estado[i])
        
        if goal[i] != estado[i]: 
        #Hago esto con conjuntos para determinar las paradas que han de hacer los ascensores Lentos
        #Por ejemplo si el inicio esta en el bloque 1 y el objetivo es el bloque3 entonces ha de hacer dos paradas
            if plantaIni < plantaFin: #Sube 
                if plantaFin in bloque2 and plantaIni in bloque1: 
                    result += 1
                elif plantaFin in bloque3 and plantaIni in bloque1:
                    result +=2
                elif plantaFin in bloque3 and plantaIni in bloque2:
                    result += 1
            else: # Baja
                if plantaIni in bloque3 and plantaFin in bloque2: 
                    result += 1
                elif plantaIni in bloque3 and plantaFin in bloque1:
                    result +=2
                elif plantaIni in bloque2 and plantaFin in bloque1:
                    result += 1
                
    return result


def hu(node): 
    return distanciaPersonaDesdeAscensor(node) + distanciaPersonaDesdePlanta(node)
    
    

    
    

In [140]:
#astar_search(p, distanciaParaObjetivo).solution()

In [195]:
astar_search(p, linear).solution()

[('Montar', 0, -1, 0, 0),
 ('Subir', 0, -1, 0, 0),
 ('Subir', 0, -1, 0, 0),
 ('Subir', 0, -1, 0, 0),
 ('Salir', 0, -1, 0, 1),
 ('Montar', 0, -1, 1, 1),
 ('Subir', 0, -1, 1, 1),
 ('Salir', 0, -1, 1, 1),
 ('Bajar', -1, -1, 1, 1),
 ('Montar', 1, -1, 1, 1),
 ('Subir', 1, -1, 1, 1),
 ('Subir', 1, -1, 1, 1),
 ('Subir', 1, -1, 1, 1),
 ('Subir', 1, -1, 1, 1),
 ('Salir', 1, -1, 1, 2)]

In [209]:
%%time
astar_search(p, distanciaPersonaDesdePlanta).solution()

KeyboardInterrupt: 